# Úkol č. 4 - regrese (do 2. ledna)

  * Cílem tohoto úkolu je vyzkoušet si řešit regresní problém na reálných (ale celkem vyčištěných) datech.
  
> **Nejdůležitější na úkolu je to, abyste udělali vše procesně správně: korektní rozdělení datasetu, ladění hyperparametrů, vyhodnocení výsledků atp.**

## Dataset

  * Zdrojem dat je list *Data* v souboru `Residential-Building-Data-Set.xlsx` na course pages (originál zde: https://archive.ics.uci.edu/ml/datasets/Residential+Building+Data+Set#).
  * Popis datasetu najdete na listu *Descriptions* ve stejném souboru.
  

## Pokyny k vypracování

  1. Rozdělte data na trénovací a testovací množinu.
  1. Proveďte základní průzkum dat a příp. vyhoďte nezajímavé příznaky.
  1. Aplikujte lineární a hřebenovou regresi a výsledky řádně vyhodnoťte:
    * K měření chyby použijte `mean_absolute_error`.
    * Experimentujte s tvorbou nových příznaků (na základě těch dostupných).
    * Experimentujte se standardizací/normalizací dat.
    * Vyberte si hyperparametry modelů k ladění a najděte jejich nejlepší hodnoty.
  1. Použijte i jiný model než jen lineární a hřebenovou regresi.


## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte pouze tento Jupyter Notebook, opravujíví by neměl nic jiného potřebovat.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_validate

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# načtení dat včetně multiindex header
data = pd.read_excel("Residential-Building-Data-Set.xlsx", header=[0,1])
# vyhození sloupců START QUARTER a COMPLETION QUARTER, protože jsou málo vypovícající a tato informace se dá zjistit na základě jiných sloupců
data = data.drop(('PROJECT DATES (PERSIAN CALENDAR)', 'START QUARTER'), axis=1)
data = data.drop(('PROJECT DATES (PERSIAN CALENDAR)', 'COMPLETION QUARTER'), axis=1)
# odtranění sloupců s informací 'Gold price per ounce'
data = data.drop(('ECONOMIC VARIABLES AND INDICES IN TIME LAG 1', 'V-29'), axis=1)
data = data.drop(('ECONOMIC VARIABLES AND INDICES IN TIME LAG 2', 'V-29'), axis=1)
data = data.drop(('ECONOMIC VARIABLES AND INDICES IN TIME LAG 3', 'V-29'), axis=1)
data = data.drop(('ECONOMIC VARIABLES AND INDICES IN TIME LAG 4', 'V-29'), axis=1)
data = data.drop(('ECONOMIC VARIABLES AND INDICES IN TIME LAG 5', 'V-29'), axis=1)

In [3]:
print('Počet nulových hodnot:', (data == 0).sum().sum())
print('Počet None hodnot:', data.isna().sum().sum())

Počet nulových hodnot: 0
Počet None hodnot: 0


Dataframe neobsahuje žádné None hodnoty ani žádné nulové hodnoty

###### 1. Rozdělte data na trénovací a testovací množinu.

In [4]:
def split_data(Xdata, ydata, ratio=0.3, rd_seed=5656):
    '''
        rozdělí pouze na trenovací a testovací, slouží k použití cross validace
    '''
    Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata, test_size=ratio, random_state=rd_seed) 
    return Xtrain, Xtest, ytrain, ytest

In [5]:
def lin_regression(Xtrain, ytrain, Xtest, ytest):
    '''provede základní lineární regresi'''
    clf = LinearRegression()
    clf.fit(Xtrain, ytrain)
    Yth = clf.predict(Xtest)
    ytest = np.array(ytest)
    for i in range(len(ytest)):
        rmse = np.sqrt(mean_absolute_error(np.array(Yth[i]), np.reshape(np.array(ytest[i]),2)))
        if rmse > 1000000:
            print(i, "rádek", Yth[i], ytest[i])
    return mean_absolute_error(Yth, ytest)

In [6]:
def ridge_regression(Xtrain, ytrain, Xtest, ytest):
    '''provede základná hřebenovou regresi s výchozími parametry'''
    clf = Ridge()
    clf.fit(Xtrain,ytrain)
    Yth = clf.predict(Xtest)
    return mean_absolute_error(Yth, np.array(ytest))

In [7]:
def get_standardized_datasets(Xtrain, Xtest):
    '''provede standardizaci dat'''
    scaler = StandardScaler() # standardizace
    scaler.fit(Xtrain)
    XStrain = scaler.transform(Xtrain)
    XStest = scaler.transform(Xtest)
    return XStrain, XStest

##### Lineární a hřebenové regrese bez standardizace a ladění hyperparametrů

In [8]:
Xtrain, Xtest, ytrain, ytest = split_data(data.drop([('OUTPUTS', 'V-9'), ('OUTPUTS', 'V-10')], axis=1), data.OUTPUTS)

In [9]:
print("RMSE lineární regrese:", lin_regression(Xtrain, ytrain, Xtest, ytest))
print("RMSE hřebenové regrese:", ridge_regression(Xtrain, ytrain, Xtest, ytest))

RMSE lineární regrese: 59.72272901755605
RMSE hřebenové regrese: 59.414088716970184


**Bez jakéhokoli ladění vychází hřebenová regrese o trochu lépe než lineární (řádově o 1%)**

##### Lineární a hřebenová regrese se standardizací příznaků

In [10]:
Xtrain, Xtest, ytrain, ytest = split_data(data.drop([('OUTPUTS', 'V-9'), ('OUTPUTS', 'V-10')], axis=1), data.OUTPUTS)
XStrain, XStest = get_standardized_datasets(Xtrain, Xtest)

In [11]:
print("RMSE lineární regrese:", lin_regression(XStrain, ytrain, XStest, ytest))
print("RMSE hřebenové regrese:", ridge_regression(XStrain, ytrain, XStest, ytest))

17 rádek [-3.14804188e+13  1.99832334e+11] [1400  300]
RMSE lineární regrese: 141429692498.51025
RMSE hřebenové regrese: 54.028082539364895


- **Po standardizaci příznaků se hřebenová regrese docela výrazně zlepšila (z 64 na 54)**
- **U lineární regrese se po standardizaci občas objevují velmi vychýlené hodnoty (vypisuji je ve výpisu)**

##### Lineární a hřebenová regrese se standardizací příznaků a s přidáním příznaků na základě sloupce s hodnotami ZIP code

In [12]:
Xtrain, Xtest, ytrain, ytest = split_data(data.drop([('OUTPUTS', 'V-9'), ('OUTPUTS', 'V-10')], axis=1), data.OUTPUTS)
tmp = Xtrain['PROJECT PHYSICAL AND FINANCIAL VARIABLES']['V-1'].astype(str).str.get_dummies()
Xtrain.drop(('PROJECT PHYSICAL AND FINANCIAL VARIABLES', 'V-1'), axis=1)
Xtrain = Xtrain.join(tmp)
tmp = Xtest['PROJECT PHYSICAL AND FINANCIAL VARIABLES']['V-1'].astype(str).str.get_dummies()
Xtest.drop(('PROJECT PHYSICAL AND FINANCIAL VARIABLES', 'V-1'), axis=1)
Xtest = Xtest.join(tmp)

D:\School\Anaconda\lib\site-packages\pandas\core\reshape\merge.py:617: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [13]:
XStrain, XStest = get_standardized_datasets(Xtrain, Xtest)

In [14]:
print("RMSE lineární regrese (standardizace):", lin_regression(XStrain, ytrain, XStest, ytest))
print("RMSE lineární regrese (bez standardizace):", lin_regression(Xtrain, ytrain, Xtest, ytest))
print("RMSE hřebenové regrese (standardizace):", ridge_regression(XStrain, ytrain, XStest, ytest))
print("RMSE hřebenové regrese (bez standardizace):", ridge_regression(Xtrain, ytrain, Xtest, ytest))

17 rádek [-2.91598959e+13 -1.22898415e+11] [1400  300]
RMSE lineární regrese (standardizace): 130726760519.72592
RMSE lineární regrese (bez standardizace): 64.37171381877694
RMSE hřebenové regrese (standardizace): 58.58906684798148
RMSE hřebenové regrese (bez standardizace): 63.387669226651525


**Závěr: toto zvýšení dimenzionality ovlivnilo Lineární i hřebenovou regresi ve všech případech negativně**

###### Vyberte si hyperparametry modelů k ladění a najděte jejich nejlepší hodnoty.

In [15]:
RANDOM_STATE = 5000 # zafixování kvůli replikovatelnosti 

In [16]:
from sklearn.model_selection import GridSearchCV, cross_val_score, ShuffleSplit
def ridge_regression_with_hyperparameters(Xtrain, ytrain, Xtest, ytest):
    params = {
        'solver' : ['svd', 'cholesky', 'lsqr', 'sparse_cg'],
        'alpha' : np.arange(0.5, 10.0, 0.5),
    }
    param_comb = ParameterGrid(params)
    results = []
    for params in param_comb:
        params['random_state'] = RANDOM_STATE
        clf = Ridge(**params)
        scores = cross_validate(clf, Xtrain, ytrain, cv=6)
        results.append(np.average(scores['test_score']))
    best_params = param_comb[np.argmax(results)]
#     print(results)
    return best_params

In [17]:
import warnings
Xtrain, Xtest, ytrain, ytest = split_data(data.drop([('OUTPUTS', 'V-9'), ('OUTPUTS', 'V-10')], axis=1), data.OUTPUTS)
XStrain, XStest = get_standardized_datasets(Xtrain, Xtest)

best_params = {}
with warnings.catch_warnings(): # ignorování FutureWarnings
    warnings.filterwarnings("ignore")
    best_params = ridge_regression_with_hyperparameters(XStrain, ytrain, XStest, ytest)
best_params['random_state'] = RANDOM_STATE
clf = Ridge(**best_params)
clf.fit(XStrain, ytrain)
Yth = clf.predict(XStest)
print('parametry:', best_params)
print('RMSE po ladění hyperparametrů (standardizace):', mean_absolute_error(Yth, np.array(ytest)))

parametry: {'solver': 'lsqr', 'alpha': 2.0, 'random_state': 5000}
RMSE po ladění hyperparametrů (standardizace): 53.894667496493824


**Celkově ladím následující hyperparametry ('solver' a 'alpha'), k ladění využívám cross validace. Model po odladění hyperparametrů je lepší pouze o trochu (cca 1% na testovacích datech)**

###### 4. Použijte i jiný model než jen lineární a hřebenovou regresi.

In [18]:
Xtrain, Xtest, ytrain, ytest = split_data(data.drop([('OUTPUTS', 'V-9'), ('OUTPUTS', 'V-10')], axis=1), data.OUTPUTS)
XStrain, XStest = get_standardized_datasets(Xtrain, Xtest)

In [19]:
def learn_forest(Xtrain, ytrain, number_of_cv):
    ''' naučení náhodného regresního lesu k odhadu lineární regrese '''
    param_grid_forest = {
        'n_estimators': range(4,40,2),
        'max_depth': range(3,8),
    }
    param_comb = ParameterGrid(param_grid_forest)
    val_acc_forest = []
    for params in param_comb:
        params['random_state'] = RANDOM_STATE
        dt = RandomForestRegressor(**params, n_jobs=4)
        scores = cross_validate(dt, Xtrain, ytrain, cv=number_of_cv, n_jobs=4) # využití cross validace k ladění hyperparametrů
        val_acc_forest.append(np.average(scores['test_score']))
    best_params = param_comb[np.argmax(val_acc_forest)]
    return best_params

In [20]:
with warnings.catch_warnings(): # ignorování FutureWarnings
    warnings.filterwarnings("ignore")
    best_params = learn_forest(XStrain, ytrain, 7)

In [21]:
dt = RandomForestRegressor(**best_params)
dt.fit(XStrain, ytrain)  # využití standardizovaných dat
Yth = dt.predict(XStest)
print('parametry:', best_params)
print('RMSE po ladění hyperparametrů:', mean_absolute_error(Yth, np.array(ytest)))

parametry: {'n_estimators': 36, 'max_depth': 7}
RMSE po ladění hyperparametrů: 74.37554934272056


**Závěr: i přes ladění hyperparametrů pomocí cross validace vychází hodnoty z regresního náhodného lesu daleko hůře než při použití základní lineární regrese**